In [5]:
import numpy as np
import math
import tensorflow
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam


In [28]:
"""
numpy version = 1.19.3
sklearn version = 0.24
tensorflow version = 2.4.0
"""

'\nnumpy version = 1.19.3\nsklearn version = 0.24\ntensorflow version = 2.4.0\n'

In [6]:
with np.load("training-dataset.npz") as data:
    img = data["x"]
    lbl = data['y'] 

lbl = lbl - 1 # the label data starts at 1 where our predict_classes will start at 0, therefore I changed lbl in order to match with the predict function

In [46]:
print(lbl)

[22  6 15 ... 12 14 18]


In [19]:
X_train, X_alt, y_train_unbin, y_alt = train_test_split(img, lbl, test_size=3/10) #documentation: https://www.researchgate.net/post/Is-there-an-ideal-ratio-between-a-training-set-and-validation-set-Which-trade-off-would-you-suggest
X_val, X_test, y_val_unbin, y_test = train_test_split(X_alt, y_alt, test_size=1/2, random_state=456) #data is split twice as we need train, val, test


lb = LabelBinarizer()
y_train = lb.fit_transform(y_train_unbin) #dummy codes are needed to train the NN
y_val = lb.transform(y_val_unbin)

X_train_val = np.vstack((X_train, X_val))
y_train_val = np.vstack((y_train, y_val))


In [31]:
print(y_test.shape)
print(X_alt.shape)
print(y_train_unbin.shape)
print(X_val.shape)

(18720,)
(37440, 784)
(87360,)
(18720, 784)


In [30]:
### Calculate the baseline accuracy, ours should be higher

baseline = LogisticRegression()
baseline.fit(X_alt, y_alt)
print(accuracy_score(y_test, baseline.predict(X_test)))

0.747542735042735


/Users/manavmishra/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [51]:
activation = 'tanh'
batch_size = 100
learning_rate = 0.0001
epochs = 300
Ni = img.shape[1]
No = 26
Ns = y_train.size
alpha = 5
beta = [0.9, 0.8]
layer_size = Ns / (alpha * (Ni + No))


model = Sequential() #Initiates the model
model.add(Dense(Ni, input_dim=img.shape[1], activation=activation)) #This is our input layer, has the same number of nodes as dimensions in our data
model.add(Dense(layer_size, activation=activation)) # This is the hidden layers
model.add(Dense(No, activation='softmax'))#Output layer. We have 26 letters, so output layers should have 26 units. Softmax is used for multi-class classification
                                                            
optimizer = Adam(lr=learning_rate, beta_1=beta[0], beta_2=beta[1])
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ["accuracy"]) #Accuracy and categorical cross-entropy is used as it's a classification problem
model.fit(X_train, y_train, epochs=100, batch_size=batch_size, verbose=1)

y_pred_train = np.argmax(model.predict(X_train), axis=-1) #Predict_classes raises an error and cannot be used soon, thus we use this instead
acc_train = round(accuracy_score(y_train_unbin, y_pred_train), 3) #We calculate the accuracy on train, test and val set and print them at the end. 


Epoch 1/100
11000/87360 [==>...........................] - ETA: 1:37 - loss: 2.1393 - acc: 0.3996- ETA: 1:39 - loss: 2.2

KeyboardInterrupt: 

In [49]:
print(layer_size)

560.8296296296296


In [38]:
y_pred_train = np.argmax(model.predict(X_train), axis=-1) #Predict_classes raises an error and cannot be used soon, thus we use this instead
acc_train = round(accuracy_score(y_train_unbin, y_pred_train), 3)
print(acc_train)

0.968


In [39]:
model = Sequential() #Initiates the model
model.add(Dense(Ni, input_dim=img.shape[1], activation=activation)) #This is our input layer, has the same number of nodes as dimensions in our data
model.add(Dense(layer_size, activation=activation)) # This is the hidden layers
model.add(Dense(No, activation='softmax'))#Output layer. We have 26 letters, so output layers should have 26 units. Softmax is used for multi-class classification
                                                            
optimizer = Adam(lr=learning_rate, beta_1=beta[0], beta_2=beta[1])
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ["accuracy"]) #Accuracy and categorical cross-entropy is used as it's a classification problem
model.fit(X_train_val, y_train_val, epochs=100,  batch_size=batch_size, verbose=1)

Epoch 1/100
106080/106080 [==============================] - 37s 353us/sample - loss: 1.1257 - acc: 0.6777
Epoch 2/100
106080/106080 [==============================] - 45s 426us/sample - loss: 0.7016 - acc: 0.7946
Epoch 3/100
106080/106080 [==============================] - 44s 418us/sample - loss: 0.5731 - acc: 0.8289
Epoch 4/100
106080/106080 [==============================] - 42s 400us/sample - loss: 0.5011 - acc: 0.8488
Epoch 5/100
106080/106080 [==============================] - 49s 457us/sample - loss: 0.4484 - acc: 0.8632
Epoch 6/100
106080/106080 [==============================] - 46s 436us/sample - loss: 0.4139 - acc: 0.8723
Epoch 7/100
106080/106080 [==============================] - 49s 458us/sample - loss: 0.3808 - acc: 0.8822
Epoch 8/100
106080/106080 [==============================] - 46s 431us/sample - loss: 0.3595 - acc: 0.8867
Epoch 9/100
106080/106080 [==============================] - 47s 440us/sample - loss: 0.3440 - acc: 0.8923
Epoch 10/100
106080/106080 [=========

In [41]:
y_pred_test = np.argmax(model.predict(X_test), axis=-1)
acc_test = round(accuracy_score(y_test, y_pred_test), 3)
print(acc_test)

0.908


In [48]:
model.evaluate(X_train,y_train)

87360/87360 [==============================] - 10s 119us/sample - loss: 0.0952 - acc: 0.9666


[0.09524973370544203, 0.96658653]